In [1]:
import numpy as np
import pandas as pd
import os
PATH = 'C:/Workspace/python/Data_Science/dataA/ProDS특강/'
file_list = os.listdir(PATH)
print(file_list)

['Dataset.zip', 'Dataset_01.csv', 'Dataset_02.csv', 'Dataset_03.csv', 'Dataset_04.csv', 'Dataset_05.csv', 'Dataset_05_item_list.csv', 'Dataset_05_Mart_POS.csv', 'Dataset_06.csv', 'Dataset_07.csv', 'Dataset_08.csv', 'Dataset_09.csv', 'Dataset_10.csv', 'Dataset_11.csv', 'Dataset_12.csv', 'Dataset_13_test.csv', 'Dataset_13_train.csv', 'Dataset_14.csv', 'Dataset_15_item_list.csv', 'Dataset_15_Mart_POS.csv']


In [95]:
df_06 = pd.read_csv(PATH + file_list[8])
print(df_06.columns)
print(f'1번 답 : {int(round(abs(df_06[df_06.waterfront == 0].price.mean() - df_06[df_06.waterfront == 1].price.mean()), 0))}')

var_list = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'yr_built']
df_06[var_list].corr().price.drop(['price'])
max, min = df_06[var_list].corr().price.drop(['price']).max(), df_06[var_list].corr().price.drop(['price']).min()
max_name = list(df_06[var_list].corr().price.drop(['price']).reset_index()[df_06[var_list].corr().price.drop(['price']).reset_index().price == max]['index'])[0]
min_name = list(df_06[var_list].corr().price.drop(['price']).reset_index()[df_06[var_list].corr().price.drop(['price']).reset_index().price == min]['index'])[0]
print(f'2번 답 : {max_name}, {min_name}')


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from statsmodels.api import OLS, add_constant

eliminate_list = ['id', 'date', 'zipcode']
df_06_test = df_06.drop(eliminate_list, axis = 1)
# lm = LinearRegression().fit(df_06_test.drop(['price'], axis = 1), df_06_test['price'])
# lm.coef_
xx = add_constant(df_06_test.drop(['price'], axis = 1))
ols1 = OLS(df_06_test['price'], xx, axis = 1).fit() # y, x 순으로 넣어야함
ols1.summary()

ols1.pvalues.drop(['const']).index[ols1.pvalues.drop(['const']) >= 0.05] # ['sqft_lot', 'sqft_lot15'] / 유의하지 않은 변수들
ols1.pvalues.drop(['const']).index[ols1.pvalues.drop(['const']) < 0.05] # 유의한 변수들
ols1_param = ols1.params[ols1.pvalues.drop(["const"]).index[ols1.pvalues.drop(["const"]) < 0.05]]
len(ols1_param[ols1_param < 0])

print(f'유의한 변수의 개수 :{len(ols1.pvalues.drop(["const"]).index[ols1.pvalues.drop(["const"]) < 0.05])}, 음수인 변수의 개수 : {len(ols1_param[ols1_param < 0])}')

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')
1번 답 : 1167273
2번 답 : sqft_living, yr_built
유의한 변수의 개수 :13, 음수인 변수의 개수 : 2


C:\Users\Bestc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [194]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
# Solver = ‘liblinear’, random_state = 12

df_07 = pd.read_csv(PATH + file_list[9])
print(df_07.columns)
var_list = ['GRE', 'TOEFL', 'CGPA', 'Chance_of_Admit']
df_07_corr = df_07[var_list].corr().Chance_of_Admit.drop(['Chance_of_Admit']).reset_index()
df_07_corr = df_07_corr.sort_values(by = 'Chance_of_Admit')
df_07_corr.reset_index(drop = True, inplace = True)
print(f'1번 답 : {round(df_07_corr.Chance_of_Admit[2], 3)}')

mu = df_07.GRE.mean()
# df_07_gre_ovav = df_07[df_07.GRE - mu >= 0] # 평균 이상
# df_07_gre_unav = df_07[df_07.GRE - mu < 0] # 평균 이하
df_07['GRE_gr'] = np.where(df_07.GRE >= mu, 1, 0)
df_07_gre_ovav = df_07[df_07['GRE_gr'] == 1].CGPA
df_07_gre_unav = df_07[df_07['GRE_gr'] == 0].CGPA

from scipy.stats import ttest_ind
df_07_out = ttest_ind(df_07_gre_ovav, df_07_gre_unav, equal_var = True)
print(df_07_out.statistic, df_07_out.pvalue)

df_07 = pd.read_csv(PATH + file_list[9])
df_07['label'] = np.where(df_07.Chance_of_Admit > 0.5, 0, 1)
df_07.drop(['Serial_No'], axis = 1, inplace = True)
from sklearn.linear_model import LogisticRegression
x_list = df_07.columns.drop(['Chance_of_Admit', 'label'])
logit = LogisticRegression(fit_intercept = False, random_state = 12, solver = 'liblinear')
logit.fit(df_07[x_list], df_07.label)
print(abs(logit.coef_).max())
df_07_out = pd.Series(logit.coef_.reshape(-1))
df_07_out.index = x_list
df_07_out.abs().nlargest(1)

Index(['Serial_No', 'GRE', 'TOEFL', 'University_Rating', 'SOP', 'LOR', 'CGPA',
       'Research', 'Chance_of_Admit'],
      dtype='object')
1번 답 : 0.873
19.443291692470982 1.1127037052806161e-59
1.9553546939533757


CGPA    1.955355
dtype: float64

In [295]:
import pandas as pd
import numpy as np
import scipy.stats as ststs
from sklearn.linear_model import LinearRegression

df_10 = pd.read_csv(PATH + file_list[-8])
var_list = ['model', 'engine_power', 'age_in_days', 'km', 'previous_owners', 'price']
df_10 = df_10[var_list]
print(df_10.columns)
df_10_ch = df_10.copy()
df_10_ch['owner_num']= np.where(df_10_ch.previous_owners <= 1, 1, 0)
df_10_ch = df_10_ch[df_10_ch.owner_num == 1]
df_10_ch = df_10_ch[df_10_ch.engine_power == 51]
df_10_ch.reset_index(drop = True, inplace = True)
q1 = pd.pivot_table(df_10_ch, index = 'model', values = ['km', 'age_in_days'])
q1['km_day'] = q1.km / q1.age_in_days
print(round(q1.km_day.min() / q1.km_day.max(), 2))

min_group = q1.km_day.idxmin() # 최솟값 그룹명
max_group = q1.km_day.idxmax() # 최댓값 그룹명

q2 = df_10.copy()
q2['km_per_day'] = q2.km / q2.age_in_days
max_km = q2[q2.model == max_group].km_per_day
min_km = q2[q2.model == min_group].km_per_day

from scipy.stats import ttest_ind
q2_out = ttest_ind(max_km, min_km, equal_var = True) # H0 : 평균차이가 없다 / H1 : 평균차이가 있다. // p-value가 0.05이상이면 귀무가설 기각
np.floor(q2_out.pvalue * 100) / 100 

q3 = df_10.copy()
q3 = q3[q3.previous_owners == 2].reset_index(drop = True)
q3 = q3[q3.model == 'pop'].reset_index(drop = True)
var_list = ['engine_power', 'age_in_days', 'km']
from sklearn.linear_model import LinearRegression
lm = LinearRegression().fit(q3[var_list], q3.price)
# lm.predict([[51, 400, 9500]])
new_data = {'engine_power' : [51], 'age_in_days' : [400], 'km' : [9500]}
lm.predict(pd.DataFrame(new_data))

Index(['model', 'engine_power', 'age_in_days', 'km', 'previous_owners',
       'price'],
      dtype='object')
0.97


array([10367.53433763])